In [ ]:
!pip install -q qiskit qiskit-nature qiskit-aer pyscf qiskit_algorithms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 12.8 MB/s eta 0:00:00


In [ ]:
# pylint: disable=line-too-long
import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_algorithms.optimizers import SLSQP,SPSA
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable
# pylint: enable=line-too-long
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
import numpy as np
import pyscf


from qiskit_nature.second_q.algorithms import ExcitedStatesEigensolver
from qiskit_algorithms import NumPyEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, QEOM, EvaluationRule
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [ ]:
from qiskit.primitives import Estimator

def get_problem():

    molecule = MoleculeInfo(
        symbols=["O","H", "H"],
        coords=([0.0, 0.0, 0.0], [0.757, 0.586, 0.0], [-0.757, 0.586, 0.0]),
        multiplicity=1,
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)


    properties = driver.run()


    problem =  ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=3).transform(properties)

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals


    mapper = ParityMapper()
    return num_particles, num_spatial_orbitals, problem, mapper



In [ ]:
optimizer = SLSQP(maxiter=10)
noiseless_estimator = Estimator()


exact_energies=[]

qeomvqe_energies=[]

def filter_criterion(eigenstate, eigenvalue, aux_values):
    return np.isclose(aux_values["ParticleNumber"][0], 2.0) and np.isclose(
        aux_values["Magnetization"][0], 0.0
    )



(num_particles, num_spatial_orbitals, problem, mapper) = get_problem()


numpy_solver = NumPyEigensolver(k=4, filter_criterion=filter_criterion)

numpy_excited_states_solver = ExcitedStatesEigensolver(mapper, numpy_solver)
numpy_results = numpy_excited_states_solver.solve(problem)

print(numpy_results)

init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
var_form = UCCSD(
    num_spatial_orbitals, num_particles, mapper, initial_state=init_state
)
vqe = VQE(
    noiseless_estimator,
    var_form,
    optimizer,
    initial_point=[0.0] * var_form.num_parameters,
)

gse = GroundStateEigensolver(mapper, vqe)

qeom_excited_states_solver = QEOM(gse, noiseless_estimator, "sd", EvaluationRule.ALL)

qeom_results = qeom_excited_states_solver.solve(problem)

# print(
#     f"Interatomic Distance: {np.round(dist, 2)}",
#     f"VQE Result: {vqe_result:.5f}",
#     f"Exact Energy: {exact_energies[-1]:.5f}",
# )

# print(
#     f"Interatomic Distance: {np.round(dist, 2)}")
# print(numpy_results.total_energies)
# print("\n\n")
# # print(qeom_results)
# print(qeom_results.total_energies)
# print("-------------------------------------")

exact_energies.append(numpy_results)
qeomvqe_energies.append(qeom_results)

print("All energies have been calculated")

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -84.158433793194
  - computed part:      -1.664218767487
  - ActiveSpaceTransformer extracted energy part: -82.494215025708
~ Nuclear repulsion energy (Hartree): 9.193913160623
> Total ground state energy (Hartree): -74.964520632571
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -83.748995300431
> Total excited state energy (Hartree): -74.555082139808
  2: 
* Electronic excited state energy (Hartree): -83.671785427766
> Total excited state energy (Hartree): -74.477872267143
  3: 
* Electronic excited state energy (Hartree): -83.648425981921
> Total excited state energy (Hartree): -74.454512821297
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 1.000 S^2: 2.000 M: 0.000
  2:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  3:  # Particles: 2.000 S: 1.000 S^2: 2.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nucle

In [ ]:
print(qeom_results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -84.158433792972
  - computed part:      -1.664218767264
  - ActiveSpaceTransformer extracted energy part: -82.494215025708
~ Nuclear repulsion energy (Hartree): 9.193913160623
> Total ground state energy (Hartree): -74.964520632348
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -83.74899427829
> Total excited state energy (Hartree): -74.555081117666
  2: 
* Electronic excited state energy (Hartree): -83.671784405587
> Total excited state energy (Hartree): -74.477871244963
  3: 
* Electronic excited state energy (Hartree): -83.648425229957
> Total excited state energy (Hartree): -74.454512069334
  4: 
* Electronic excited state energy (Hartree): -83.599724875481
> Total excited state energy (Hartree): -74.405811714857
  5: 
* Electronic excited state energy (Hartree): -83.0604391128
> Total excited state energy (Hartree): -73.866525952176
  6: 
* Electronic excited state ene